# TensorFlow 2.x quick start for beginners
[Notebook](https://www.tensorflow.org/tutorials/quickstart/beginner)<br>

Imports

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


Data are loaded and normalized to reduce calculation times

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [7]:
tf.reduce_min(x_train), tf.reduce_max(x_train)

(<tf.Tensor: shape=(), dtype=float64, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float64, numpy=1.0>)

## Build a machinelearning model

**Flatten** is used to flatten the matrix into a line, with the pixels placed side by side in a row.<br>
**Dense** Layer of 128 neurons.<br>
**Dropout** Prevents overlearning by preventing neurons from becoming too specialized or too dependent on other specific neurons in the network. By randomly "forgetting" certain neurons during training, Dropout forces the network to learn more robust and generalizable features.<br>

The last layer contains 10 neurons, as there are 10 different classes in Mnist

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [13]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.43123165, -0.57455575,  0.89089406, -0.5928584 , -0.21564117,
         0.800264  ,  0.17173916, -0.0855756 ,  0.39184996,  1.3964509 ]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class:

In [14]:
tf.nn.softmax(predictions).numpy()

array([[0.04371971, 0.03788195, 0.16400953, 0.03719492, 0.05423841,
        0.14979902, 0.0798995 , 0.06177229, 0.09957195, 0.27191278]],
      dtype=float32)

Define a loss function for training using losses.SparseCategoricalCrossentropy:

In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [17]:
loss_fn(y_train[:1], predictions).numpy()


1.8984609

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [18]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])


## Train and evaluate your model
Use the Model.fit method to adjust your model parameters and minimize the loss:

In [19]:
model.fit(x_train, y_train, epochs=5)


Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2959 - accuracy: 0.9149
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1460 - accuracy: 0.9567
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1103 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0883 - accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0732 - accuracy: 0.9773


The Model.evaluate method checks the model's performance, usually on a validation set or test set.

In [20]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0806 - accuracy: 0.9760 - 604ms/epoch - 2ms/step


[0.08063695579767227, 0.9760000109672546]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [21]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [22]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.0392599e-07, 5.1624607e-09, 2.1602732e-07, 1.2150577e-05,
        2.5218364e-11, 2.1091228e-08, 3.6262979e-13, 9.9998224e-01,
        2.3564041e-08, 5.1553593e-06],
       [3.4515256e-08, 1.2726113e-04, 9.9986398e-01, 7.8512294e-06,
        1.5712316e-15, 2.5496905e-07, 1.9176149e-08, 1.3574050e-13,
        5.8341959e-07, 7.2691107e-12],
       [5.9698748e-09, 9.9778301e-01, 1.0651491e-04, 6.5737900e-06,
        4.5262575e-05, 2.3887701e-06, 5.6000849e-06, 1.5123042e-03,
        5.3761766e-04, 7.3405863e-07],
       [9.9947995e-01, 2.7282917e-08, 8.3187027e-05, 3.6697378e-07,
        4.0413721e-07, 2.9647742e-06, 1.5743756e-04, 1.9019921e-04,
        2.2263879e-07, 8.5230764e-05],
       [6.9573889e-06, 1.4105530e-08, 9.6061876e-06, 3.2996201e-07,
        9.7649914e-01, 1.1453670e-06, 1.6204718e-05, 3.8442659e-04,
        4.4615641e-05, 2.3037512e-02]], dtype=float32)>